## Ensemble의 Ensemble 실습
- 대게 Ensemble 단일 기법에 Bagging Ensemble을 섞으면 성능이 좋아지는 경향이 있음.

In [1]:
import pandas as pd
import numpy as np
import random
import lightgbm as lgb

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('./data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [3]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [16]:
bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 리스트로 학습 데이터 인덱스 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 복원 추출(Bagging 기법)
    print(len(set(random_data_index))) # 복원 추출된 리스트 개수 확인
    lgb_dtrain = lgb.Dataset(data=train_x.iloc[random_data_index,], label=train_y.iloc[random_data_index,]) # 복원 추출된 학습 데이터를 LightBGM 형식으로 변환
    lgb_params = {'max_depth': 15,
                  'learning_rate': 0.01,
                  'n_estimators': 500,
                  'objective': 'regression',
                  'num_leaves': 32,
                  'force_col_wise': True} # 하이퍼 파라미터 지정(튜닝 가능)
    lgb_model = lgb.train(params=lgb_params, train_set=lgb_dtrain)
    predict = lgb_model.predict(test_x)
    bagging_predict_result.append(predict)

9552
[LightGBM] [Info] Total Bins 231
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 534254.987507
9628
[LightGBM] [Info] Total Bins 237
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 539655.805010
9580
[LightGBM] [Info] Total Bins 232
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 533332.344438
9664
[LightGBM] [Info] Total Bins 235
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 541266.848305
9530
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 15129, number of used features: 8
[LightGBM] [Info] Start training from score 539413.216604
9564
[LightGBM] [Info] Total Bins 238
[LightGBM] [Info] Numb

In [17]:
bagging_predict_result

[array([509347.11013766, 594938.19898108, 897636.04748974, ...,
        333877.61798466, 890528.59352329, 465110.88984015]),
 array([539205.34988028, 673102.75875544, 911343.90414919, ...,
        342234.57864796, 935898.27920499, 462611.29703546]),
 array([515358.35683222, 607005.52182275, 985186.44025219, ...,
        354584.78832351, 937828.73665651, 443514.71073052]),
 array([544102.06037801, 601459.37010768, 917154.30782425, ...,
        339432.1515591 , 883369.7702682 , 473376.7132515 ]),
 array([ 499619.11549826,  588209.13365805,  869194.97927529, ...,
         351308.1499368 , 1011599.27415378,  463785.16261294]),
 array([529249.84770133, 620238.92964986, 920232.00863336, ...,
        342367.73215751, 942733.669402  , 468756.36549997]),
 array([469656.9147615 , 607079.72686567, 967766.88241753, ...,
        344918.27443618, 903617.0135624 , 466485.69131199]),
 array([494318.43463541, 605446.47507473, 956799.69886176, ...,
        333681.68718096, 826996.75992714, 483261.135710

In [18]:
# 예측한 결과값을 바탕으로 평균을 계산
bagging_predict = []
for lst2_index in range(test_x.shape[0]):
    temp_predict = []
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging한 결과 예측값을 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 인덱스 10개의 결과값에 대한 평균을 최종 리스트에 저장

In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [20]:
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y))))

RMSE: 209909.86240280553


In [21]:
bagging_predict

[514870.1138795597,
 613995.5950311035,
 930701.833888138,
 1615781.3974181863,
 638988.5332350159,
 367576.2562889494,
 702399.9117124924,
 435474.3023203801,
 463081.1019145093,
 493678.31545660377,
 631814.5722612415,
 380854.98229172634,
 301378.9542804389,
 359247.37255053583,
 342792.60437266814,
 1337269.2995762134,
 366936.10968688567,
 1008476.398866786,
 319657.1305666401,
 528020.019193135,
 372083.9947961708,
 1824912.8422231234,
 677496.9891269079,
 516046.1441932899,
 504057.8293435541,
 487437.15595229773,
 297379.63860646344,
 244797.13730721353,
 474369.88247783424,
 525262.0389545287,
 484862.09419064305,
 468574.1089468788,
 461938.9906967981,
 590935.1356205461,
 385058.2795879911,
 1018583.784047549,
 889982.2082206846,
 528037.7983060761,
 354465.0574215305,
 1537454.2666931576,
 395702.7976752251,
 276371.9094839694,
 511404.9061342158,
 343663.54921021743,
 253710.52550909139,
 238778.79100229387,
 333369.13458347536,
 334117.8682066088,
 352910.9269502134,
 553